In [1]:
import os
os.chdir("../..")

In [2]:
import torch
from utils.dataset_loader import load_datasets
from utils.model_utils import initialize_model
from utils.train_utils import train_model
from utils.metrics import evaluate_model
from utils.visualization import plot_training, plot_confusion_matrix

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
data_dir = "wildfire_dataset_scaled"
batch_size = 32
num_epochs = 15
learning_rate = 0.001
num_classes = 2

In [5]:
print("[INFO] Loading datasets...")
train_loader, val_loader, test_loader = load_datasets(data_dir, batch_size, augmentation="baseline")
print("[INFO] Datasets loaded successfully!")



[INFO] Loading datasets...
[INFO] Datasets loaded successfully!


In [6]:
print("\n[INFO] Starting Baseline Training for Model: resnet18\n")
print("[INFO] Initializing ResNet18 model...")
model = initialize_model(
    model_name="convnext_tiny",
    num_classes=num_classes,
    pretrained=True,
    freeze_all=True  # Baseline: Freeze all layers
)
print("[INFO] Model initialized successfully!")


[INFO] Starting Baseline Training for Model: resnet18

[INFO] Initializing ResNet18 model...


C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Yash\Documents\Development\Projects\final_year\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[INFO] Model initialized successfully!


In [7]:
print("[INFO] Setting up loss function and optimizer...")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
print("[INFO] Loss function and optimizer set up!")

[INFO] Setting up loss function and optimizer...
[INFO] Loss function and optimizer set up!


In [ ]:
print("[INFO] Starting model training...")
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,  # No learning rate scheduler for baseline
    num_epochs=num_epochs,
    device=device,
    save_path="outputs/models/baseline/convnext_tiny.pt",
    early_stop_patience=5,     # Stop after 5 epochs without improvement
    monitor_metric="val_recall",
)
print("[INFO] Training completed successfully!")

[INFO] Starting model training...

Starting training...



Epoch [1/15]:
    Train Loss: 0.3924, Train Acc: 0.8288
    Val Loss:   0.3117, Val Acc:   0.8706
    Learning Rate: 0.001000



Epoch [2/15]:
    Train Loss: 0.2762, Train Acc: 0.8861
    Val Loss:   0.2692, Val Acc:   0.9005
    Learning Rate: 0.001000



Epoch [3/15]:
    Train Loss: 0.2381, Train Acc: 0.8988
    Val Loss:   0.2522, Val Acc:   0.9080
    Learning Rate: 0.001000



Epoch [4/15]:
    Train Loss: 0.2267, Train Acc: 0.9094
    Val Loss:   0.2634, Val Acc:   0.8980
    Learning Rate: 0.001000



...

Epoch [11/15]:
    Train Loss: 0.1582, Train Acc: 0.9422
    Val Loss:   0.2161, Val Acc:   0.9179
    Learning Rate: 0.001000



Epoch [12/15]:
    Train Loss: 0.1590, Train Acc: 0.9406
    Val Loss:   0.2115, Val Acc:   0.9179
    Learning Rate: 0.001000



Epoch [13/15]:
    Train Loss: 0.1553, Train Acc: 0.9396
    Val Loss:   0.2143, Val Acc:   0.9179
    Learning Rate: 0.001000



Epoch [14/15]:
    Train Loss: 0.1543, Train Acc: 0.9422
    Val Loss:   0.2119, Val Acc:   0.9154
    Learning Rate: 0.001000



Epoch [15/15] - Training:  81%|██████████████████████████████████████████████▎          | 48/59 [00:04<00:01, 10.50it/s, Batch Loss=0.1169]

In [ ]:
print("[INFO] Evaluating the model...")
metrics = evaluate_model(
    model=model,
    test_loader=test_loader,
    classes=["No Fire", "Fire"],
    device=device
)
print(f"\n[INFO] Metrics for ResNet18:\n{metrics}")

In [ ]:
print("[INFO] Saving training and evaluation results...")
plot_training(history, "outputs/resnet18_baseline_training_curve.png")
plot_confusion_matrix(
    cm=metrics["confusion_matrix"],
    classes=["No Fire", "Fire"],
    output_path="outputs/resnet18_baseline_confusion_matrix.png"
)
print("[INFO] Results saved successfully!")